# [75.06 / 95.58] Organización de Datos <br> Trabajo Práctico 2: Machine Learning
# Notebook Principal

**Grupo 30: Datatouille**

- 101055 - Bojman, Camila
- 100029 - del Mazo, Federico
- 100687 - Hortas, Cecilia
- 97649 - Souto, Rodrigo

**http://fdelmazo.github.io/7506-Datos/**

**https://www.kaggle.com/datatouille2018/competitions**

Continuando la investigación sobre la empresa Trocafone realizada en el [TP1](https://fdelmazo.github.io/7506-Datos/TP1/TP1.html), se busca determinar la probabilidad de que un usuario del sitio realice una conversión en el período determinado.

Notebooks en orden de corrida y lectura:

0. [TP1](https://fdelmazo.github.io/7506-Datos/TP1/TP1.html) --> Familiarización con el set de datos y exploración de estos.

1. [Investigación Previa](https://fdelmazo.github.io/7506-Datos/TP2/investigacion.html) --> Con ayuda de lo trabajado en el TP1, se averiguan más cosas de las datos, en busqueda de que poder reutilizar.

2. [Creación de Dataframes](https://fdelmazo.github.io/7506-Datos/TP2/new-dataframes.html) --> Como parte del feature engineering, se crean dataframes nuevos con información de los productos del sitio y de como se accede a este (marcas, sistemas operativos, etc).

3. [Feature Engineering](https://fdelmazo.github.io/7506-Datos/TP2/feature-engineering.html) --> Busqueda de atributos de los usuarios de los cuales se busca predecir la conversión

4. [Submission Framework](https://fdelmazo.github.io/7506-Datos/TP2/submission-framework.html) --> Pequeño framework para construir las postulaciones de labels. 

5. Algoritmos de clasificación (directorio `AlgoritmosClasificacion`) --> Definiciones de algoritmos de machine learning para utilizar en este notebook

  * [Árboles de decisión](https://fdelmazo.github.io/7506-Datos/TP2/AlgoritmosClasificacion/decision-tree.html): TODO, descr
  * [Random Forest](https://fdelmazo.github.io/7506-Datos/TP2/AlgoritmosClasificacion/random-forest.html): TODO, descr
  * [XGBoost](https://fdelmazo.github.io/7506-Datos/TP2/AlgoritmosClasificacion/xgboost.html): TODO, descr

6. TP2 --> Teniendo todo en cuenta, juntar todos los datos buscados y encontrados sobre un mismo dataframe, aplicarle todos los algoritmos definidos previamente e ir realizando los entrenamientos y posteriores predicciones de conversiones con las que se arman las postulaciones.

In [ ]:
# Set-up inicial, se deja comentado para evitar instalarle módulos al usuario

In [ ]:
## Primero, descargar los datasets de no tenerlos

# Antes de comenzar, buscar las credenciales de kaggle (el nombre de usuario, y el token) y setearlos en las siguientes dos lineas
# %env KAGGLE_USERNAME="datatouille2018"
# %env KAGGLE_KEY="xxx"

# !pip install kaggle # https://github.com/Kaggle/kaggle-api
# !kaggle competitions download -c trocafone -p data
# !unzip -q data/events_up_to_01062018.csv.zip -d data
# !rm data/events_up_to_01062018.csv.zip
# !ls data/

In [ ]:
# # Luego, descargar los módulos a utilizar a lo largo de todo el trabajo

# !pip install nbimporter
# !conda install -c conda-forge xgboost 

In [ ]:
import nbimporter
import pandas as pd
import calendar
from sklearn.model_selection import train_test_split
import os.path

from AlgoritmosClasificacion import *
import submission_framework as SF

In [ ]:
df_users = pd.read_csv('data/user-features.csv',low_memory=False).set_index('person')
df_y = pd.read_csv('data/labels_training_set.csv').groupby('person').sum()

In [ ]:
df_users.head()

## Algoritmos de Machine Learning

---

### Random Forest

---

### Decision Tree


---

### XGBoost


---
---

## Lista de robos

TODO: Borrar

* [ ] https://github.com/urielkelman/abracadata/tree/master/TP2

* [ ] https://github.com/GastonMontes/Datos-TP2

* [ ] https://github.com/MatiasReimondo/Datos

* [ ] Ver diapos argerich feature engineering

* [ ] Spammear ramos mejia de preguntas

* [ ] Notebooks de Martin

* [ ] Notebook de Juan

## Lista de cosas a hacer

* [ ] Dividir en set de entrenamientos y set de test. Ojo, hay que hacerlo con tiempo! No se puede hacer al azar. Los primeros meses entrenan a los siguientes. Etc

* [ ] Identificar bias y varianza, ploteando error de set de entrenamiento y error de set de test en funcion de cantidad de datos en set de entrenamiento (mismo plot)

* [ ] Perturbar datos de entrada -> reducir overfitting

* [ ] Feature engineering --> no olvidar documentar y versionar!!!

* [ ] Catboost

* [ ] Feature: dia de la semana

* [ ] Loopear interacciones importantes! Random forest claveee

* [ ] No perder nombre de feature original

* [ ] Scrapear trocafone --> Precios

* [ ] Xgboost no sirve para importancia de features. No es estable

* [ ] Reclamar 50 usd aws &#128544; 

* [ ] Clustering para nuevos features para entrenar